In [23]:
import pandas as pd

data = pd.read_csv("df.csv")
data_corr = data.drop(columns=["Company", "Sector"])
data_corr_upr = data_corr.corr()
print(data_corr_upr)

# Největší vliv na indikátor akcie Y má ukazatel FL - finanční páka.

           CR        DA        FL       OPM        RT       TAT        DE  \
CR   1.000000 -0.225917 -0.179232  0.284529 -0.151203 -0.123665 -0.141136   
DA  -0.225917  1.000000  0.340320  0.015032  0.206620 -0.083691  0.464786   
FL  -0.179232  0.340320  1.000000 -0.112430 -0.076335  0.156169  0.968612   
OPM  0.284529  0.015032 -0.112430  1.000000  0.051672 -0.298701 -0.050014   
RT  -0.151203  0.206620 -0.076335  0.051672  1.000000  0.206580 -0.065267   
TAT -0.123665 -0.083691  0.156169 -0.298701  0.206580  1.000000  0.094869   
DE  -0.141136  0.464786  0.968612 -0.050014 -0.065267  0.094869  1.000000   
Y   -0.018690  0.164599  0.759323  0.051308 -0.046290  0.301222  0.724604   
PT  -0.040483 -0.120993  0.032588  0.021409  0.097708  0.017872 -0.025348   

            Y        PT  
CR  -0.018690 -0.040483  
DA   0.164599 -0.120993  
FL   0.759323  0.032588  
OPM  0.051308  0.021409  
RT  -0.046290  0.097708  
TAT  0.301222  0.017872  
DE   0.724604 -0.025348  
Y    1.000000 -0.0013

In [24]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

print(res.summary())





                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     22.52
Date:                Mon, 13 Nov 2023   Prob (F-statistic):           1.45e-18
Time:                        00:13:43   Log-Likelihood:                -375.94
No. Observations:                 100   AIC:                             769.9
Df Residuals:                      91   BIC:                             793.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0176      5.347     -2.061      0.0

In [25]:
formula = "Y ~ FL + OPM + TAT"
mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     59.51
Date:                Mon, 13 Nov 2023   Prob (F-statistic):           7.99e-22
Time:                        00:13:43   Log-Likelihood:                -377.99
No. Observations:                 100   AIC:                             764.0
Df Residuals:                      96   BIC:                             774.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4188      2.865     -3.637      0.0

In [26]:
data['Y_predicted'] = res.predict(data)
data["rozdil"] = data["Y_predicted"]- data["Y"]
print(data)






   Company    CR    DA     FL     OPM      RT   TAT     DE      Y     PT  \
0    2Q3DU  1.19  0.20   2.21  0.0770    6.66  0.40   0.43   1.95   5.93   
1    46JNq  0.29  0.64  10.14  0.2037   20.04  0.36   6.52   7.26  43.48   
2    4ISOt  2.93  0.04   1.43  0.3055    6.55  0.72   0.06   7.78  18.38   
3    5AWwu  1.03  0.25   3.55  0.0708  146.43  1.80   0.88   6.17   5.15   
4    5hbZR  1.20  0.47   0.00  0.1677   30.92  0.93   0.00   0.00  17.06   
..     ...   ...   ...    ...     ...     ...   ...    ...    ...    ...   
95   yhCH3  1.50  0.37   2.33  0.2557    4.92  0.41   0.85   5.18   6.83   
96   YqdX2  0.70  0.27   2.57  0.2363   16.11  0.64   0.69   7.40   2.70   
97   yUjyz  1.49  0.12   2.36  0.2576    8.64  0.51   0.28   5.99   7.59   
98   YX515  2.30  0.64  25.03  0.3758    7.48  0.31  16.08  41.57  10.54   
99   zCRGR  1.40  0.21   2.35  0.2391    7.08  0.45   0.50   3.35   5.53   

                    Sector  Y_predicted     rozdil  
0              Industrials     0.5

In [27]:

print(data.iloc[data["rozdil"].idxmax()])

# Potenciálně nejvíce podhodnocenou akcií je akcie společnosti SYXLy.

Company              SYXLy
CR                    0.76
DA                    0.37
FL                   18.64
OPM                 0.0631
RT                   12.44
TAT                   0.41
DE                    6.93
Y                     6.97
PT                     3.4
Sector         Industrials
Y_predicted      41.779545
rozdil           34.809545
Name: 79, dtype: object


In [28]:
# Bonus 1


import statsmodels.api as sm

data_x = data[["Y", "FL", "OPM", "TAT"]]

data_x = sm.add_constant(data_x.drop(columns=["Y"]))

rlm_model = sm.RLM(data["Y"], data_x)
rlm_results = rlm_model.fit()

print(rlm_results.summary())





                    Robust linear Model Regression Results                    
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       96
Method:                          IRLS   Df Model:                            3
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Mon, 13 Nov 2023                                         
Time:                        00:13:44                                         
No. Iterations:                    50                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.7581      1.822     -3.709      0.0

In [29]:
data["Y_predicted"] = rlm_results.predict(data_x)
undervalued_stock = data.loc[data["Y_predicted"].idxmax()]
print(undervalued_stock)

# V případě robustní regese je podhodnocená akcie jiná, než v případě regresního modelu. Jedná se o akcii společnosti vwxyQ.

Company                    vwxyQ
CR                          1.19
DA                          0.47
FL                         32.52
OPM                       0.1077
RT                           0.0
TAT                         1.92
DE                         15.16
Y                          90.79
PT                          5.51
Sector         Consumer Cyclical
Y_predicted            79.527582
rozdil                   0.10132
Name: 88, dtype: object


In [30]:
# Bonus 2

from statsmodels.stats.outliers_influence import OLSInfluence

formula = "Y ~ CR + DA + RT + DE + PT + FL + OPM + TAT"
mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()
influence = OLSInfluence(res)
data["Cook_Distance"] = influence.cooks_distance[0]

print(res.summary())
print(data["Cook_Distance"])



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     22.52
Date:                Mon, 13 Nov 2023   Prob (F-statistic):           1.45e-18
Time:                        00:13:44   Log-Likelihood:                -375.94
No. Observations:                 100   AIC:                             769.9
Df Residuals:                      91   BIC:                             793.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0176      5.347     -2.061      0.0

In [31]:
outliers_cook = data["Cook_Distance"] > 1
data_no_outliers_cook = data.loc[~outliers_cook]

mod_no_outliers_cook = smf.ols(formula=formula, data=data_no_outliers_cook)
res_no_outliers_cook = mod_no_outliers_cook.fit()
print(res_no_outliers_cook.summary())



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     24.76
Date:                Mon, 13 Nov 2023   Prob (F-statistic):           1.06e-19
Time:                        00:13:44   Log-Likelihood:                -367.74
No. Observations:                  99   AIC:                             753.5
Df Residuals:                      90   BIC:                             776.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.0524      5.364     -1.128      0.2

In [32]:
undervalued_stock_no_outliers_cook = data_no_outliers_cook.loc[data_no_outliers_cook["Y"].idxmax()]
print(undervalued_stock_no_outliers_cook)

# potenciálně nejvíce podhodnocenou akcií, v případě vyloučení outliners, je akcie společnosti jhFdg.

Company                       jhFdg
CR                             1.09
DA                             0.48
FL                             24.7
OPM                          0.1913
RT                            13.43
TAT                            1.16
DE                             11.9
Y                            109.24
PT                             4.76
Sector           Consumer Defensive
Y_predicted               59.145578
rozdil                   -42.540694
Cook_Distance              0.396365
Name: 45, dtype: object


In [33]:
# Bonus 3

import patsy

# One-Hot Encode 
data_encoded = patsy.dmatrix("0 + Sector", data, return_type='dataframe')

data_combined = pd.concat([data, data_encoded], axis=1)

# Original model with One-Hot Encoded 'Sector'
formula = "Y ~ CR + DA + RT + DE + PT + FL + OPM + TAT + Sector"
mod = smf.ols(formula=formula, data=data_combined)
res = mod.fit()
print(res.summary())

# koeficienty pro každý sektor¨
sector_coefficients = res.params.filter(like='Sector')

positive_sectors = sector_coefficients[sector_coefficients > 0].index
negative_sectors = sector_coefficients[sector_coefficients < 0].index

# Sektory s kladným koeficientem: Consumer Cyclical, Financial Services, Technology
# Sektory se záporným koeficientem: Communication Services, Conglomerates, Consumer Defensive, Energy, Healthcare, Industrials

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     12.04
Date:                Mon, 13 Nov 2023   Prob (F-statistic):           8.25e-16
Time:                        00:13:44   Log-Likelihood:                -367.95
No. Observations:                 100   AIC:                             771.9
Df Residuals:                      82   BIC:                             818.8
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep